# Advanced Dataframes

In this lesson we will continue working with pandas DataFrames, and explore some more complex DataFrame manipulations.

In [36]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [37]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [38]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [39]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


## Creating Dataframes

### From Lists and Dictionaries

There are several ways to create dataframes, we've already seen how we can create a dataframe from a dictionary:

In [40]:
pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

,A,B
0,1,4
1,2,5
2,3,6


The keys in the passed dictionary will be the column names, and the values are the data points that make up each column.

We can also create dataframes from a 2d data structure, either a numpy array or a list of lists:

In [41]:
pd.DataFrame([[1, 2, 3], [4, 5, 6 ]])

,0,1,2
0,1,2,3
1,4,5,6


In [42]:
array = np.array([[1, 2, 3], [4, 5, 6]])

pd.DataFrame(array, columns=['this', 'that', 'the other'])

,this,that,the other
0,1,2,3
1,4,5,6


Notice here that we had to specify the names of the columns ourselves.

### From PyDataset.



In some of the exercises, you'll need to load several datasets using the `pydataset` library. (If you get an error when trying to run the import below, use `pip` to install the `pydataset` package.) When the instructions say to load a dataset from PyDataset, you will need to do the following:

The following import is necessary to access PyDataset datasets:
```python
from pydataset import data
```

Running this code snippet will show you the valuable information doc on the dataset:
```python
data(df_string_name, show_doc=True)
```

Running this code snippet will load the dataset for use as a pandas DataFrame:
```python
df = data(df_string_name)
```

There are 757 available datasets using pydataset. Running the following code snippet in a cell will return a DataFrame with all of your options:
```python
data()
```

In [43]:
# Load the dataset and store it in the variable mpg.

from pydataset import data
mpg = data('mpg')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


You can view the documentation for the dataset, and any pydata dataset, by setting show_doc to True. This outputs valuable context for your dataset.

```python
data('mpg', show_doc=True) 
```

### From SQL



We can use the `read_sql` method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted *connection string*.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a **driver**, a bit of software that handles the details of the database connection.

In order to connect to mysql, we'll install the `pymysql` driver packages:

`python -m pip install pymysql`

Once those are installed, we can create the connection string. In general, database connection urls will have this format:

```python
protocol://[user[:password]@]hostname/[database_name]
```

Here's an example of what one would look like:

```python
mysql+pymysql://codeup:p@assw0rd@123.123.123.123/some_db
```

Another thing we need to consider is that we don't want to publish our database credentials to github, however, we will need access to these values in our code in order to create the connection string defined above.

In order to accomplish this, we can define several variables in a file named `env.py` that contain the sensitive data, add `env.py` to our `.gitignore` file, and then import those values into another script. 

**Be 100% sure to add `env.py` to this specific repository's `.gitignore` file, even and especially, if you have already added `env.py` to your global .gitignore file. This will protect the env file for people who clone this project (like collaborators)**

In [44]:
# pd.read_sql(query, url)

In [45]:
from env import host, username, password
import pandas as pd
url = f'mysql+pymysql://{username}:{password}@{host}/employees'

Once this url is defined, we can use it with the `read_sql` function to have pandas treat the results of a SQL query as a dataframe.

In [46]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


It is common to have longer SQL queries that we want to read into python, and an example of how we might break a query into several lines is below:

In [47]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
WHERE gender = 'F'
LIMIT 100
'''

employees = pd.read_sql(sql, url)
employees.head()

,emp_no,first_name,last_name
0,10002,Bezalel,Simmel
1,10006,Anneke,Preusig
2,10007,Tzvetan,Zielinski
3,10009,Sumant,Peac
4,10010,Duangkaew,Piveteau


!!!danger "Passwords and Sensitive Information"
    Don't add and commit files with passwords or other sensitive information in them to a git repository!

In [48]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
LIMIT 100;
'''

title_dept = pd.read_sql(query, url)
title_dept.head()

,title,dept_name
0,Staff,Customer Service
1,Senior Staff,Customer Service
2,Staff,Customer Service
3,Senior Staff,Customer Service
4,Staff,Customer Service


In [49]:
title_dept.to_csv('./temp.csv')

## Exercises I

Run `python -m pip install pymysql` from your terminal to install pymysql.

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Run `python -m pip install pymysql` from your terminal to install the mysql client (any folder is fine)
1. cd into your exercises folder for this module and run `echo env.py >> .gitignore`
1. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

2. Use your function to obtain a connection to the `employees` database.

3. Once you have successfully run a query:

    a. Intentionally make a typo in the database url. What kind of error message do you see?
    
    b. Intentionally make an error in your SQL query. What does the error message look like?

4. Read the `employees` and `titles` tables into two separate DataFrames.

5. How many rows and columns do you have in each DataFrame? Is that what you expected?

6. Display the summary statistics for each DataFrame.

7. How many unique titles are in the `titles` DataFrame?

8. What is the oldest date in the `to_date` column? 

9. What is the most recent date in the `to_date` column?

In [ ]:
# url = f'mysql+pymysql://{user}:{password}@{host}/employees'

# 1. Run `python -m pip install pymysql` from your terminal to install the mysql client (any folder is fine)
# 1. cd into your exercises folder for this module and run `echo env.py >> .gitignore`
# 1. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

'''  get_db_url()

Examples:
# Using the env = argument

get_db_url('employees',env='./env.py')

# Using positional arguments

from env import host, username, password
get_db_url('employees',username,password,host)

'''

def get_db_url(database, username='', password='', hostname='', env=''):
    if env != '':
        d = {}
        file = open(env)
        for line in file:
            (key, value) = line.split('=')
            d[key] = value.replace('\n', '').replace("'",'').replace('"','')
        username = d['username']
        hostname = d['host']
        password = d['password']
    url = f'mysql+pymysql://{username}:{password}@{hostname}/{database}'
    return url

# print( get_db_url('employees',env='./env.py') )
# print( get_db_url('employees',username,password,host) )


In [51]:

# 2. Use your function to obtain a connection to the `employees` database.
url = get_db_url('employees',env='./env.py')
statement = 'Select * from employees limit 10;'
pd.read_sql( statement, url ) 


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [52]:

# 3. Once you have successfully run a query:
#     a. Intentionally make a typo in the database url. What kind of error message do you see?
#     b. Intentionally make an error in your SQL query. What does the error message look like?
badurl = get_db_url('employees',username,'wrongpass',host)


pd.read_sql( statement, badurl ) 
    # OperationalError: (1045, "Access denied for user 'kalpana_1811'@'pool-96-253-112-221.rcmdva.fios.verizon.net' (using password: YES)")
    # The above exception was the direct cause of the following exception:


OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'kalpana_1811'@'pool-96-253-112-221.rcmdva.fios.verizon.net' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
badstatement = 'Select * limit 10;'
pd.read_sql( badstatement, url ) 

# OperationalError: (1096, 'No tables used')

# The above exception was the direct cause of the following exception:

OperationalError: (pymysql.err.OperationalError) (1096, 'No tables used')
[SQL: Select * limit 10;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# 4. Read the `employees` and `titles` tables into two separate DataFrames.
employees_statement = 'select * from employees'
titles_statement = 'select * from titles'

df_emp = pd.read_sql(employees_statement, url)
df_t = pd.read_sql(titles_statement, url)


In [ ]:
# 5. How many rows and columns do you have in each DataFrame? Is that what you expected?
print('emp:', df_emp.shape)
print('title:', df_t.shape)


emp: (300024, 6)
title: (443308, 4)


In [ ]:
# 6. Display the summary statistics for each DataFrame.
print( df_emp.describe(include = 'all').to_string() )
print( df_t.describe(include = 'all').to_string() )

# 7. How many unique titles are in the `titles` DataFrame?
## 7



               emp_no  birth_date first_name last_name  gender   hire_date
count   300024.000000      300024     300024    300024  300024      300024
unique            NaN        4750       1275      1637       2        5434
top               NaN  1952-03-08     Shahab      Baba       M  1985-06-20
freq              NaN          95        295       226  179973         132
mean    253321.763392         NaN        NaN       NaN     NaN         NaN
std     161828.235540         NaN        NaN       NaN     NaN         NaN
min      10001.000000         NaN        NaN       NaN     NaN         NaN
25%      85006.750000         NaN        NaN       NaN     NaN         NaN
50%     249987.500000         NaN        NaN       NaN     NaN         NaN
75%     424993.250000         NaN        NaN       NaN     NaN         NaN
max     499999.000000         NaN        NaN       NaN     NaN         NaN
               emp_no     title   from_date     to_date
count   443308.000000    443308      443308 

In [ ]:
# 8. What is the oldest date in the `to_date` column? 
df_t['to_date'] = pd.to_datetime( df_t['to_date'], errors='coerce' )
df_t.describe(include='datetime')


## 	1985-03-01 00:00:00

# 9. What is the most recent date in the `to_date` column?

## 2002-08-01 00:00:00 

C:\Users\Crux\AppData\Local\Temp\ipykernel_13160\2891353627.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_t.describe(include='datetime')


,to_date
count,203184
unique,5887
top,1998-10-25 00:00:00
freq,91
first,1985-03-01 00:00:00
last,2002-08-01 00:00:00


In [56]:
df_t.dtypes

emp_no                int64
title                object
from_date            object
to_date      datetime64[ns]
dtype: object

-------

## Indexing and Subsetting

Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator `[]`.

I will demonstrate concrete examples of indexing using the indexing operator `[]` alone and with the `.loc` and `.iloc` attributes below.

### `[]`

I can pass a list of columns from a DataFrame to the indexing operator (aka bracket notation) to return a subset of my original DataFrame.

In [60]:
# Choose only two columns for my subset.
columns = ['name', 'classroom']
df[columns].head()

df[['name', 'classroom']].head()

,name,classroom
0,Sally,A
1,Jane,B
2,Suzie,A
3,Billy,B
4,Ada,A


In [57]:
# I can pass a boolean Series to the indexing operator as a selector.

bools = df.name.str.startswith('A')
bools

0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9     False
10    False
11     True
Name: name, dtype: bool

In [58]:
df[bools]

,name,math,english,reading,classroom
4,Ada,77,92,98,A
8,Albert,92,62,87,A
11,Alan,92,62,72,A


### `.loc`

We can use the `.loc` attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! **The .loc attribute's indexing is inclusive and uses an index label, not position.**

```python
df.loc[row_indexer, column_indexer]
```

In [61]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [74]:
df.loc[1,'math']

88

In [64]:
# Select all the rows and a subset of columns; notice the inclusive behavior of the indexing.

df.loc[0:4, 'math':'reading']

,math,english,reading
0,62,85,80
1,88,79,67
2,94,74,95
3,98,96,88
4,77,92,98


### `.iloc`

We can use the `.iloc` attribute to select specific rows and colums by index position. `.iloc` does not accept a boolean Series as a selector like `.loc` does. **It takes in integers representing index position and is NOT inclusive.**

```python
df.iloc[row_indexer, column_indexer]
```

We can select rows by integer position:

In [65]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A


We can also specify which columns we want to select:

In [66]:
df.iloc[:3, 1:3]

,math,english
0,62,85
1,88,79
2,94,74


Here we select the first 3 rows (everything up to but not including the index of 3), and the second and third columns (starting from the index of 1 up to but not including the index of 3).

## Aggregating

### `.agg`

The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [67]:
df.math.median()

90.0

In [68]:
df.math.min()

62

In [69]:
df.reading.agg('min')

67

While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

In [70]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [71]:
df[['english', 'reading', 'math']].mean()

english    77.666667
reading    86.500000
math       84.833333
dtype: float64

In [72]:
df[['english', 'reading', 'math']].agg(['mean', 'min', 'max'])

,english,reading,math
mean,77.666667,86.5,84.833333
min,62.000000,67.0,62.000000
max,99.000000,98.0,98.000000


### `.groupby`

The `.groupby` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [ ]:
df[["classroom", "math", "reading", "english"]].groupby("classroom").max()

In [ ]:
df.groupby('classroom').math.max()

We can use `.agg` here to, to see multiple aggregations:

In [ ]:
df.groupby('classroom').math.agg(['min', 'mean', 'max'])

In [ ]:
df.head()

We can group by multiple columns as well. To demonstrate, we'll create a boolean column named `passing_math`, then group by the combination of our new feature, `passing_math`, and the classroom and calculate the average reading grade and the number of individuals in each subgroup. 

Let's break this problem down and code it step-by-step.

#### `np.where`

First, we can create the new `passing_math` column using a handy NumPy function called `np.where`. It will allow us to base the new column values on whether the values in an existing column, `math`, meet a condition.

```python
np.where(condition, this_where_True, this_where_False)
```

In [79]:
# Create the new column based on an existing column.

df['passing_math'] = np.where(df.math < 70, False, True)
df

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,False
1,Jane,88,79,67,B,True
2,Suzie,94,74,95,A,True
3,Billy,98,96,88,B,True
4,Ada,77,92,98,A,True
5,John,79,76,93,B,True
6,Thomas,82,64,81,A,True
7,Marie,93,63,90,A,True
8,Albert,92,62,87,A,True
9,Richard,69,80,94,A,False


Next, we will group by the `passing_math` and `classroom` columns and use the `.agg` method to calculate the average reading grade and the number of students.

In [81]:
grade_groups = df.groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

mean  count
passing_math classroom                  
False        A          87.000000      2
True         A          87.166667      6
             B          85.250000      4

In [82]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups

avg_reading_grade  count_of_students
passing_math classroom                                      
False        A                  87.000000                  2
True         A                  87.166667                  6
             B                  85.250000                  4

**Takeaways:**

We can interpret this output as there being 2 students failing math in classroom A with an average reading grade of 87, 6 students passing math in classroom A with an average reading grade of 87.16, and 4 students passing math in classroom B with an average reading grade of 85.25.

## Merging and Joining


Pandas provides several ways to combine dataframes together. We will look at two of them below:

### `pd.concat`

This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.



For example, concat with a list of two DataFrames
```python
pd.concat([df1, df2], axis=0)
```

- When your list contains at least one DataFrame, a DataFrame is returned.


- When concatenating only Series objects row-wise, axis=0, a Series is returned.


- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

Default is set to row-wise concatenation using an outer join.
```python
pd.concat(objs, axis=0, join='outer')
```

When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names[^1].

In [83]:
"con" + "cat" + "e" + "nation"

'concatenation'

In [84]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1

,a
0,1
1,2
2,3


In [85]:
df2

,a
0,4
1,5
2,6


In [86]:
pd.concat([df1, df2])

,a
0,1
1,2
2,3
0,4
1,5
2,6


**Note** that the indices are preserved on the resulting dataframe; we could set the `ignore_index` parameter to `True` if we wanted these to be sequential.

In [87]:
concat_df1 = pd.concat([df1, df2], ignore_index=True)
concat_df1

,a
0,1
1,2
2,3
3,4
4,5
5,6


[^1]:
    We can concatenate dataframes with different column names, but generally this is not the behavior we want, as pandas will fill in a lot of null values into the resulting dataframe. The exception to this is if the dataframes are aligned on their index (i.e. the labels for each row), then we can provide the `axis=1` keyword argument to `pd.concat` to merge the dataframes horizontally.

In [88]:
concat_df2 = pd.DataFrame({'b': [1, 2, 3, 4, 5, 6]})
concat_df2

,b
0,1
1,2
2,3
3,4
4,5
5,6


In [89]:
# Axis=1 means concat horizontally
pd.concat([concat_df1, concat_df2], axis=1)

,a,b
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6


In [91]:
# pd.concat([df1, df2])
pd.concat([pd.concat([df1, df2]), concat_df2], axis=1, ignore_index=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

### `.merge`

This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.



`df.merge` default settings for commonly used parameters.

```python
left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
```

How does changing the default argument of the `how` parameter change my resulting DataFrame?

`how` == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [92]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [93]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


The `.merge` method will allow us to specify `left_on` and `right_on` to indicate the columns that are the keys used to merge the dataframes together. 

- In addition, the `how` keyword argument is used to define what type of JOIN we want to do; as we saw above, `inner` is the default setting. 

- For demonstration purposes, I'm also going to set the `indicator` parameter to `True`, which will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [96]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of `_x` to any columns in the left dataframe that are duplicated, and `_y` to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:

In [98]:
# Inner join tolerates no nulls.
# Inner join only shows user who have roles, roles that have users
users.merge(roles, how="inner", left_on="role_id", right_on="id", indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1,bob,1.0,1,admin,both
1,2,joe,2.0,2,author,both
2,3,sally,3.0,3,reviewer,both
3,4,adam,3.0,3,reviewer,both


In [ ]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)

## Exercises II

1. Copy the `users` and `roles` DataFrames from the examples above. 

2. What is the result of using a `right` join on the DataFrames? 

3. What is the result of using an `outer` join on the DataFrames?
     
4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

5. Load the `mpg` dataset from PyDataset. 

6. Output and read the documentation for the `mpg` dataset.

7. How many rows and columns are in the dataset?

8. Check out your column names and perform any cleanup you may want on them.

9. Display the summary statistics for the dataset.

10. How many different manufacturers are there?

11. How many different models are there?

12. Create a column named `mileage_difference` like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

13. Create a column named `average_mileage` like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

14. Create a new column on the `mpg` dataset named `is_automatic` that holds boolean values denoting whether the car has an automatic transmission.

15. Using the `mpg` dataset, find out which which manufacturer has the best miles per gallon on average?

16. Do automatic or manual cars have better miles per gallon?

In [99]:
# 1. Copy the `users` and `roles` DataFrames from the examples above. 
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

users, roles


(   id   name  role_id
 0   1    bob      1.0
 1   2    joe      2.0
 2   3  sally      3.0
 3   4   adam      3.0
 4   5   jane      NaN
 5   6   mike      NaN,
    id       name
 0   1      admin
 1   2     author
 2   3   reviewer
 3   4  commenter)

In [101]:
# 2. What is the result of using a `right` join on the DataFrames? 
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)

## Only the users that have a role are listed, and all roles are listed even if no user has that role.

,id,employee,role_id,role
0,1.0,bob,1,admin
1,2.0,joe,2,author
2,3.0,sally,3,reviewer
3,4.0,adam,3,reviewer
4,NaN,NaN,4,commenter


In [103]:

# 3. What is the result of using an `outer` join on the DataFrames?
users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer',
            indicator=True)

## All rows are returned from both tables. NaN values are filled for any cell that doesn't have a match.

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [106]:
# 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?
(users
    .drop(columns='role_id')
    .merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
)

## Python produces an error: "Key Error"


KeyError: 'role_id'

In [118]:
# 5. Load the `mpg` dataset from PyDataset. 
from pydataset import data
mpg = data('mpg')
mpg.head()


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [117]:
# 6. Output and read the documentation for the `mpg` dataset.
data('mpg', True)


mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [110]:
# 7. How many rows and columns are in the dataset?
mpg.shape



(234, 11)

In [114]:
# 8. Check out your column names and perform any cleanup you may want on them.
mpg.columns


Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [115]:
# 9. Display the summary statistics for the dataset.
mpg.info(), mpg.describe(include='all')



<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


(None,
        manufacturer        model       displ         year         cyl  \
 count           234          234  234.000000   234.000000  234.000000   
 unique           15           38         NaN          NaN         NaN   
 top           dodge  caravan 2wd         NaN          NaN         NaN   
 freq             37           11         NaN          NaN         NaN   
 mean            NaN          NaN    3.471795  2003.500000    5.888889   
 std             NaN          NaN    1.291959     4.509646    1.611534   
 min             NaN          NaN    1.600000  1999.000000    4.000000   
 25%             NaN          NaN    2.400000  1999.000000    4.000000   
 50%             NaN          NaN    3.300000  2003.500000    6.000000   
 75%             NaN          NaN    4.600000  2008.000000    8.000000   
 max             NaN          NaN    7.000000  2008.000000    8.000000   
 
            trans  drv         cty         hwy   fl class  
 count        234  234  234.000000  234.000

In [125]:
# 10. How many different manufacturers are there?
mpg.groupby('manufacturer')['model'].agg(['count'])
mpg.groupby('manufacturer')['model'].agg(['count']).size

## 15

15

In [158]:
# 11. How many different models are there?
# len( mpg['model'].unique() )
mpg.model.nunique()

## 38


38

In [ ]:
# 12. Create a column named `mileage_difference` like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.
mpg['mileage_difference'] = np.diff(mpg[['cty','hwy']], axis=1)

mask = mpg['mileage_difference'] != (mpg.hwy - mpg.cty)
mpg[mask]


In [150]:

# 13. Create a column named `average_mileage` like you did in the DataFrames exercises; this is the mean of the city and highway mileage.
# mpg['average_mileage'] = (mpg.city + mpg.highway) / 2
mpg['average_mileage']= mpg[['cty','hwy']].mean(axis=1)


In [157]:
mpg.head(2)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,is_automatic,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,True,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,False,8,25.0


In [152]:
# 14. Create a new column on the `mpg` dataset named `is_automatic` that holds boolean values denoting whether the car has an automatic transmission.
mask = np.where(mpg.trans.str.contains('auto'), True, False)
mpg[mask].head()

mpg['is_automatic'] = np.where(mpg.trans.str.contains('auto'), True, False)
mpg.head(3)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,is_automatic,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,True,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,False,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,False,11,25.5


In [155]:

# 15. Using the `mpg` dataset, find out which which manufacturer has the best miles per gallon on average?
mpg.groupby('manufacturer')['average_mileage'].agg(['mean']).sort_values('mean',ascending=False)

## Honda



,mean
manufacturer,
honda,28.500000
volkswagen,25.074074
hyundai,22.750000
subaru,22.428571
audi,22.027778
toyota,21.720588
pontiac,21.700000
nissan,21.346154
chevrolet,18.447368


In [156]:
# 16. Do automatic or manual cars have better miles per gallon?
mpg.groupby('is_automatic')['average_mileage'].agg(['mean']).sort_values('mean',ascending=False)

## Manual

,mean
is_automatic,
False,22.227273
True,19.130573


## Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.

### `pd.crosstab`

For an example of `.crosstab`, we will count the number of students passing math in each classroom.

In [159]:
# We will use our student grades DataFrame, df.

df

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,False
1,Jane,88,79,67,B,True
2,Suzie,94,74,95,A,True
3,Billy,98,96,88,B,True
4,Ada,77,92,98,A,True
5,John,79,76,93,B,True
6,Thomas,82,64,81,A,True
7,Marie,93,63,90,A,True
8,Albert,92,62,87,A,True
9,Richard,69,80,94,A,False


We'll use the `pd.crosstab` function to count the number of occurances of each subgroup (i.e. each unique combination of classroom and whether or not the student is passing math):

In [160]:
pd.crosstab(df.passing_math, df.classroom)

classroom,A,B
passing_math,,
False,2,0
True,6,4


We can also view subtotals with the `margins` set to `True`.

In [161]:
pd.crosstab(df.passing_math, df.classroom, margins=True)

classroom,A,B,All
passing_math,,,
False,2,0,2
True,6,4,10
All,8,4,12


The `.crosstab` function will let us view the numbers as percentages of the total as well by setting `normalize` to `True`.

In [162]:
pd.crosstab(df.passing_math, df.classroom, normalize=True).round(3)

classroom,A,B
passing_math,,
False,0.167,0.000
True,0.500,0.333


In [164]:
pd.crosstab(df.passing_math, df.classroom, normalize='columns').round(3)

classroom,A,B
passing_math,,
False,0.25,0.0
True,0.75,1.0


### `.pivot_table`

Here we use the `.pivot_table` method to create our summary. This method produces output similar to an excel pivot table. We must supply 3 things here:

- which values will make up the rows (the `index`)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`); if we can omit this, and `mean` will be used by default

For an example using the `pivot_table` method, we'll calculate the average math grade for the combination of `classroom` and `passing_math` status.

Here we'll create a dataframe that represents various orders at a restaurant.

In [165]:
df.pivot_table(index='classroom', columns='passing_math', values='math')

passing_math,False,True
classroom,,
A,65.5,88.333333
B,NaN,89.250000


In [166]:
df.pivot_table(index='classroom', columns='passing_math', values='math',aggfunc='median')

passing_math,False,True
classroom,,
A,65.5,92.0
B,NaN,90.0


In [167]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)

,drink,meal
0,Tea,Pad Thai
38,Water,Curry
11,Tea,Curry
4,Tea,Curry
16,Water,Curry
2,Tea,Yakisoba Noodle
22,Water,Yakisoba Noodle
5,Water,Pad Thai
19,Tea,Pad Thai
15,Water,Yakisoba Noodle


#### `.map`

The `.map` method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named `bill`. Let's do this step-by-step.

In [168]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

In [169]:
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

orders.sample(10)

,drink,meal,bill
6,Water,Pad Thai,10
31,Water,Curry,11
7,Water,Yakisoba Noodle,9
32,Water,Yakisoba Noodle,9
0,Tea,Pad Thai,12
22,Water,Yakisoba Noodle,9
38,Water,Curry,11
15,Water,Yakisoba Noodle,9
21,Tea,Yakisoba Noodle,11
29,Tea,Yakisoba Noodle,11


Let's take a look at how many orders have each combination of meal and drink:

In [170]:
pd.crosstab(orders.drink, orders.meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,3,4,5
Water,9,10,9


In [171]:
pd.crosstab(orders.drink, orders.meal, normalize=True, margins=True)

meal,Curry,Pad Thai,Yakisoba Noodle,All
drink,,,,
Tea,0.075,0.10,0.125,0.3
Water,0.225,0.25,0.225,0.7
All,0.300,0.35,0.350,1.0


And let's find out the average bill amount for each combination: 

In [172]:
orders.pivot_table(index='drink', columns='meal', values='bill')

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


It's interesting to note that we could find the same information with a multi-level group by:

In [173]:
orders.groupby(['drink', 'meal']).bill.mean()

drink  meal           
Tea    Curry              13.0
       Pad Thai           12.0
       Yakisoba Noodle    11.0
Water  Curry              11.0
       Pad Thai           10.0
       Yakisoba Noodle     9.0
Name: bill, dtype: float64

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand. 

### Transposing

In [178]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11
name,Sally,Jane,Suzie,Billy,Ada,John,Thomas,Marie,Albert,Richard,Isaac,Alan
math,62,88,94,98,77,79,82,93,92,69,92,92
english,85,79,74,96,92,76,64,63,62,80,99,62
reading,80,67,95,88,98,93,81,90,87,94,93,72
classroom,A,B,A,B,A,B,A,A,A,A,B,A
passing_math,False,True,True,True,True,True,True,True,True,False,True,True


In [177]:
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11
name,Sally,Jane,Suzie,Billy,Ada,John,Thomas,Marie,Albert,Richard,Isaac,Alan
math,62,88,94,98,77,79,82,93,92,69,92,92
english,85,79,74,96,92,76,64,63,62,80,99,62
reading,80,67,95,88,98,93,81,90,87,94,93,72
classroom,A,B,A,B,A,B,A,A,A,A,B,A
passing_math,False,True,True,True,True,True,True,True,True,False,True,True


In [175]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
math,12.0,84.833333,11.134168,62.0,78.50,90.0,92.25,98.0
english,12.0,77.666667,13.371158,62.0,63.75,77.5,86.75,99.0
reading,12.0,86.500000,9.643651,67.0,80.75,89.0,93.25,98.0


## Exercises III

1. Use your `get_db_url` function to help you explore the data from the `chipotle` database. 

2. What is the total price for each order?

3. What are the most popular 3 items?

4. Which item has produced the most revenue?

5. Join the `employees` and `titles` DataFrames together.

6. For each title, find the hire date of the employee that was hired most recently with that title.

7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [180]:
url = get_db_url('chipotle',env='./env.py')
statement='select * from orders'
chipotle = pd.read_sql(statement,url)
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [182]:
chipotle.info(), chipotle.describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


(None,
                      count unique           top  freq        mean  \
 id                  4622.0    NaN           NaN   NaN      2311.5   
 order_id            4622.0    NaN           NaN   NaN  927.254868   
 quantity            4622.0    NaN           NaN   NaN    1.075725   
 item_name             4622     50  Chicken Bowl   726         NaN   
 choice_description    4622   1044           nan  1246         NaN   
 item_price            4622     78        $8.75    730         NaN   
 
                             std  min      25%     50%      75%     max  
 id                  1334.400802  1.0  1156.25  2311.5  3466.75  4622.0  
 order_id             528.890796  1.0   477.25   926.0   1393.0  1834.0  
 quantity               0.410186  1.0      1.0     1.0      1.0    15.0  
 item_name                   NaN  NaN      NaN     NaN      NaN     NaN  
 choice_description          NaN  NaN      NaN     NaN      NaN     NaN  
 item_price                  NaN  NaN      NaN     NaN   

In [ ]:
# 2. What is the total price for each order?
def unstring_usd(str):
    return float(str.replace('$','').replace(',',''))
chipotle['item_price'] = chipotle['item_price'].apply(unstring_usd)


In [187]:
chipotle['total_price'] = chipotle.item_price * chipotle.quantity
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price,total_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,33.96


In [192]:
def as_currency(amount):
    if amount >= 0:
        return '${:,.2f}'.format(amount)
    else:
        return '-${:,.2f}'.format(-amount)
chipotle.groupby('order_id')['total_price'].sum().apply(as_currency)

order_id
1       $11.56
2       $33.96
3       $12.67
4       $21.00
5       $13.70
         ...  
1830    $23.00
1831    $12.90
1832    $13.20
1833    $23.50
1834    $28.75
Name: total_price, Length: 1834, dtype: object

In [196]:
# 3. What are the most popular 3 items?
chipotle.groupby('item_name')['quantity'].sum().nlargest(3)


item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

In [223]:
# 4. Which item has produced the most revenue?
chipotle.groupby('item_name')['item_price'].sum().nlargest(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

In [199]:

# 5. Join the `employees` and `titles` DataFrames together.
url = get_db_url('employees',env='./env.py')
employees = pd.read_sql(employees_statement, url)
titles = pd.read_sql(titles_statement, url)
employees.head(), titles.head()


(   emp_no  birth_date first_name last_name gender   hire_date
 0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
 1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
 2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
 3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
 4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12,
    emp_no            title   from_date     to_date
 0   10001  Senior Engineer  1986-06-26  9999-01-01
 1   10002            Staff  1996-08-03  9999-01-01
 2   10003  Senior Engineer  1995-12-03  9999-01-01
 3   10004         Engineer  1986-12-01  1995-12-01
 4   10004  Senior Engineer  1995-12-01  9999-01-01)

In [202]:

emp_title = employees.merge(titles,
                            how='outer',
                            left_on='emp_no',
                            right_on='emp_no')
emp_title.head()


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [204]:
# 6. For each title, find the hire date of the employee that was hired most recently with that title.
emp_title.groupby('title')['hire_date'].agg(['max'])



,max
title,
Assistant Engineer,1999-12-24
Engineer,2000-01-28
Manager,1992-02-05
Senior Engineer,2000-01-01
Senior Staff,2000-01-13
Staff,2000-01-12
Technique Leader,1999-12-31


In [207]:
# 7. Write the code necessary to create a cross tabulation of the number of titles by department. 
# (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)
emp_url = get_db_url('employees',env='./env.py')
dept_emp_statement = '''
    SELECT 
        *
    FROM
        employees.dept_emp AS demp
    JOIN
        employees.departments AS dep USING(dept_no)
    '''
dept_emp = pd.read_sql(dept_emp_statement, emp_url)
dept_emp.head()

,dept_no,emp_no,from_date,to_date,dept_name
0,d009,10011,1990-01-22,1996-11-09,Customer Service
1,d009,10038,1989-09-20,9999-01-01,Customer Service
2,d009,10049,1992-05-04,9999-01-01,Customer Service
3,d009,10060,1992-11-11,9999-01-01,Customer Service
4,d009,10088,1992-03-21,9999-01-01,Customer Service


In [209]:
tdemp = pd.merge(titles,dept_emp, how='inner', left_on='emp_no', right_on='emp_no')
tdemp.head()

,emp_no,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y,dept_name
0,10001,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01,Development
1,10002,Staff,1996-08-03,9999-01-01,d007,1996-08-03,9999-01-01,Sales
2,10003,Senior Engineer,1995-12-03,9999-01-01,d004,1995-12-03,9999-01-01,Production
3,10004,Engineer,1986-12-01,1995-12-01,d004,1986-12-01,9999-01-01,Production
4,10004,Senior Engineer,1995-12-01,9999-01-01,d004,1986-12-01,9999-01-01,Production


In [218]:
import datetime
mask1 = tdemp.to_date_x > datetime.date.today()
mask2 = tdemp.to_date_y > datetime.date.today()
tdemp_current = tdemp[(mask1) & (mask2)]
tdemp_current.to_date_x.min()

datetime.date(9999, 1, 1)

In [220]:
pd.crosstab(tdemp.title, tdemp.dept_name).T.sort_index()

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,298,2362,4,2027,13925,16150,309
Development,7769,58135,2,49326,1247,1424,7683
Finance,0,0,2,0,12139,13929,0
Human Resources,0,0,2,0,12274,14342,0
Marketing,0,0,2,0,13940,16196,0
Production,6445,49649,4,42205,1270,1478,6557
Quality Management,1831,13852,4,11864,0,0,1795
Research,378,2986,2,2570,11637,13495,393
Sales,0,0,2,0,36191,41808,0


In [221]:
pd.crosstab(tdemp_current.title, tdemp_current.dept_name).T.sort_index()

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,68,627,1,1790,11268,3574,241
Development,1652,14040,1,38816,1085,315,5477
Finance,0,0,1,0,9545,2891,0
Human Resources,0,0,1,0,9824,3073,0
Marketing,0,0,1,0,11290,3551,0
Production,1402,12081,1,33625,1123,349,4723
Quality Management,389,3405,1,9458,0,0,1293
Research,77,830,1,2250,9092,2870,321
Sales,0,0,1,0,28797,8903,0


### Extra Pandas Exercises and Resources
- [https://www.w3resource.com/python-exercises/pandas/index.php](https://www.w3resource.com/python-exercises/pandas/index.php)
- [https://towardsdatascience.com/20-pandas-functions-that-will-boost-your-data-analysis-process-f5dfdb2f9e05](https://towardsdatascience.com/20-pandas-functions-that-will-boost-your-data-analysis-process-f5dfdb2f9e05)
- [https://github.com/guipsamora/pandas_exercises](https://github.com/guipsamora/pandas_exercises)
- [https://github.com/ajcr/100-pandas-puzzles](https://github.com/ajcr/100-pandas-puzzles)

### More Practice!

For even more practice with pandas, you can do the exercises from the SQL module, but instead of using SQL to do the aggregation, sorting, joining, etc, use pandas. That is, read the data from all of the tables into pandas dataframes and manipulate the dataframes.